In [1]:
import rbm
import rbm_sy
import numpy as np
import projectLib as lib
import matplotlib.pyplot as plt
import inspect

In [2]:
F=5
K=5

In [3]:
training = lib.getTrainingData()
ratingsForUser1 = lib.getRatingsForUser(1, training)

In [4]:
# ratingsPerMovie(training)
v = rbm.getV(ratingsForUser1)
# v.shape
w = rbm.getInitialWeights(v.shape[0],F,5)

In [5]:
trStats = lib.getUsefulStats(training)
W = rbm.getInitialWeights(trStats["n_movies"], F, K)

In [6]:
num_movies=trStats["n_movies"]
vis_bias=np.zeros((num_movies,5))
hid_bias=np.zeros((F,))

In [7]:
h = [0,1,0,1,0]

In [8]:
rbm.hiddenToVisible(h, W[ratingsForUser1[:, 0], :, :])
h = [0,1,0,1,0]
rbm_sy.hiddenToVisible(h, W[ratingsForUser1[:, 0], :, :])

array([[0.16437925, 0.17803374, 0.22953033, 0.17622554, 0.25183114],
       [0.19279955, 0.18326502, 0.21703691, 0.23967913, 0.16721939],
       [0.22071227, 0.17724028, 0.18073978, 0.17708796, 0.24421971],
       [0.22043061, 0.19885265, 0.14141267, 0.22007137, 0.2192327 ],
       [0.23544336, 0.18986391, 0.18322699, 0.20464002, 0.18682572],
       [0.18739654, 0.24596974, 0.21513575, 0.17209598, 0.17940198],
       [0.22942796, 0.20534646, 0.2028281 , 0.18791995, 0.17447752],
       [0.18034893, 0.19204282, 0.22027372, 0.24782909, 0.15950545]])

array([[0.16437925, 0.17803374, 0.22953033, 0.17622554, 0.25183114],
       [0.19279955, 0.18326502, 0.21703691, 0.23967913, 0.16721939],
       [0.22071227, 0.17724028, 0.18073978, 0.17708796, 0.24421971],
       [0.22043061, 0.19885265, 0.14141267, 0.22007137, 0.2192327 ],
       [0.23544336, 0.18986391, 0.18322699, 0.20464002, 0.18682572],
       [0.18739654, 0.24596974, 0.21513575, 0.17209598, 0.17940198],
       [0.22942796, 0.20534646, 0.2028281 , 0.18791995, 0.17447752],
       [0.18034893, 0.19204282, 0.22027372, 0.24782909, 0.15950545]])

In [11]:
rbm.getPredictedDistribution(v, W[ratingsForUser1[:, 0], :, :], W[1, :, :], vis_bias, hid_bias)
rbm_sy.getPredictedDistribution(v, W[ratingsForUser1[:, 0], :, :], W[1, :, :])

array([0.20073695, 0.19108352, 0.19967294, 0.21294541, 0.19556118])

array([0.1576574 , 0.18611418, 0.20203868, 0.23611   , 0.21807975])